### Hyperparameter tuning with grid search

In [ ]:
import torch
from torch.utils.data import DataLoader
from core.util.get_datasets import get_park_dataset
from sklearn.model_selection import ParameterGrid
from core.models import LSTM
from core.models.model_training import train_model
from core.util.trefor_dataset import TreforData
from core.util.hyperparameter_configuration import get_hyperparameter_configuration
from core.util.early_stop import EarlyStop

ModuleNotFoundError: No module named 'core.models.LSTM'

Set ML model, loss function and hyperparameters that that will be tested.

In [ ]:
gridsearch_params = {
    "learning_rate": [0.001, 0.005],
    "batch_size": [16, 32, 64],
    "num_layers": [1, 2],
}

# Extract hyperparameters configuration that will not be tuned upon
hyperparameter_configuration = get_hyperparameter_configuration()
EPOCHS = hyperparameter_configuration["epochs"]
lookback = hyperparameter_configuration["lookback"]
loss_function = hyperparameter_configuration["loss_function"]
horizon = hyperparameter_configuration["horizon"]
hidden_size = hyperparameter_configuration["hidden_size"]
dropout_rate = hyperparameter_configuration["dropout_rate"]
model_used = LSTM

Use CUDA (GPU) if available.

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

Train a model with specified hyperparameters

In [ ]:
def train_with_params(params: dict) -> tuple[float, LSTM]:
    """Train model with the specified hyperparameters."""
    # Extract hyperparameters
    batch_size = params["batch_size"]
    learning_rate = params["learning_rate"]
    num_layers = params["num_layers"]

    # Load the data and split into training and validation sets
    x_train, y_train, x_val, y_val, _, _ = get_park_dataset(
        lookback=lookback, lookahead=horizon
    )
    x_train = torch.tensor(x_train).float()
    y_train = torch.tensor(y_train).float()
    x_val = torch.tensor(x_val).float()
    y_val = torch.tensor(y_val).float()
    train_dataset = TreforData(x_train, y_train, device)
    val_dataset = TreforData(x_val, y_val, device)

    # Create data loaders
    training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = model_used(
        input_size=x_train.shape[-1],
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout_rate=dropout_rate,
    ).to(device)

    _, val_loss, best_model = train_model(
        epochs=EPOCHS,
        model=model,
        loss_function=loss_function,
        training_loader=training_loader,
        validation_loader=validation_loader,
        learning_rate=learning_rate,
        early_stopper=EarlyStop(5, 0.05),
    )

    return min(val_loss), best_model

Iterate over all hyperparameters and train a model for each combination.

In [ ]:
best_loss = float("inf")
best_params = None
best_model = None

for params in ParameterGrid(gridsearch_params):
    v_loss, model = train_with_params(params)
    print(params, v_loss)
    if v_loss < best_loss:
        best_loss = v_loss
        best_params = params
        best_model = model

print("Best Hyperparameters:", best_params)
print("Best Validation Loss:", best_loss)